In [ ]:
def get_test_config_details(lst, dataset_condition):
    ckpt_name_list = lst[1].split('/')
    #print(ckpt_name_list)
    train, mic_idx = ckpt_name_list[-3], ckpt_name_list[-1][-2]

    exp_name_list = lst[-1].split('_')
    #idx = 4 if exp_name_list[-1][0]=='-' else 3
    if dataset_condition=="noisy_reverb":
        snr, t60, test = exp_name_list[-1][:-3], exp_name_list[-3], exp_name_list[-5]
    elif dataset_condition=="reverb":
        snr, t60, test = None, exp_name_list[-1][:-1], exp_name_list[-3]
    elif dataset_condition=="noisy":
        snr, t60, test = exp_name_list[-1][:-3], None, exp_name_list[-3]
    else:
        pass

    return mic_idx, train, test, t60, snr

In [ ]:
# getting metrics for noisy_reverb
import os
from collections import namedtuple

metrics = namedtuple('metrics', ["mix_stoi", "enh_stoi", "mix_pesq_nb", "enh_pesq_nb"])
doa_metrics = namedtuple('doa_metrics', ["mix_frm_Acc", "est_frm_Acc", "mix_blk_Acc", "est_blk_Acc"])

dataset_condition = "noisy"
comparison_domain = "doa"

scenarios = ["stationary", "moving"]
#t60_list = [0.2, 0.4, 0.6, 0.8, 1.0] # rows 
#snr_list = [-5.0, 0.0, 5.0, 10.0, 20.0] # col

files_list = os.listdir(f'../Logs/{dataset_condition}/{comparison_domain}')
exp_dict = {}
doa_exp_dict = {}
for file in files_list:
    if os.path.isfile(os.path.join(f'../Logs/{dataset_condition}/{comparison_domain}', file)) and os.path.splitext(file)[1]==".out":
        with open(os.path.join(f'../Logs/{dataset_condition}/{comparison_domain}', file), 'r') as f:
            for line in f.readlines():
                if "ckpt_dir" in line.strip():
                    mic_idx, train, test, t60, snr = get_test_config_details(line.split(), dataset_condition)
                elif "PESQ_NB_epoch" in line.strip():
                    if "PESQ_NB_epoch"==line.strip().split()[1]:
                        pesq_nb_enh = line.strip().split()[3]
                        #print(pesq_nb_enh)
                    elif "MIX_PESQ_NB_epoch"==line.strip().split()[1]:
                        pesq_nb_mix = line.strip().split()[3]
                        #print(pesq_nb_mix)
                    else:
                        continue
                    #print(line)
                elif "STOI_epoch" in line.strip():
                    if "STOI_epoch"==line.strip().split()[1]:
                        stoi_enh = line.strip().split()[3]
                        #print(stoi_enh)
                    elif "MIX_STOI_epoch"==line.strip().split()[1]:
                        stoi_mix = line.strip().split()[3]
                        #print(stoi_mix)
                    else:
                        continue
                elif "blk_Acc_epoch" in line.strip():
                    if "est_blk_Acc_epoch"==line.strip().split()[1]:
                        est_blk_Acc = line.strip().split()[3]
                    elif "mix_blk_Acc_epoch"==line.strip().split()[1]:
                        mix_blk_Acc = line.strip().split()[3]
                    else:
                        continue
                elif "frm_Acc_epoch" in line.strip():
                    if "est_frm_Acc_epoch"==line.strip().split()[1]:
                        est_frm_Acc = line.strip().split()[3]
                    elif "mix_frm_Acc_epoch"==line.strip().split()[1]:
                        mix_frm_Acc = line.strip().split()[3]
                    else:
                        continue
                
                #print(line)

    
        
        if comparison_domain =="doa":
            exp_metrics = doa_metrics(mix_frm_Acc, est_frm_Acc, mix_blk_Acc, est_blk_Acc)
        else:
            exp_metrics = metrics(stoi_mix, stoi_enh, pesq_nb_mix, pesq_nb_enh)

        #exp_metrics = { "mix_stoi": stoi_mix, "enh_stoi": stoi_enh, "mix_pesq_nb": pesq_nb_mix, "enh_pesq_nb": pesq_nb_enh}

        if dataset_condition =="reverb":
            exp_app_str = f't60_{t60}'
        elif dataset_condition =="noisy":
            exp_app_str = f'snr_{snr}'
        elif dataset_condition =="noisy_reverb":
            exp_app_str = f't60_{t60}_snr_{snr}'
        else:
            exp_app_str = ''

        key_str = f'train_{train}_mic_idx_{mic_idx}_test_{test}_{exp_app_str}' if comparison_domain!="doa" else f'train_{train}_test_{test}_{exp_app_str}'
        exp_dict[key_str] = exp_metrics
        #doa_exp_dict[f'train_{train}_mic_idx_{mic_idx}_test_{test}_{exp_app_str}'] = doa_metrics

        #break




In [ ]:

#import torch
#torch.save(exp_dict, f"{dataset_condition}_doa_results.pt")


In [ ]:
mic_idx=0
#dataset_condition = "reverb"
train="stationary"
test = "stationary"
t60=0.8
snr = -5.0
if dataset_condition =="reverb":
    exp_app_str = f't60_{t60}'
elif dataset_condition =="noisy":
    exp_app_str = f'snr_{snr}'
elif dataset_condition =="noisy_reverb":
    exp_app_str = f't60_{t60}_snr_{snr}'
else:
    exp_app_str = ''

key_str = f'train_{train}_mic_idx_{mic_idx}_test_{test}_{exp_app_str}' if comparison_domain!="doa" else f'train_{train}_test_{test}_{exp_app_str}'
exp_dict[key_str]

In [ ]:
exp_dict

In [ ]:
len(exp_dict)

In [ ]:
import torch
dataset_condition = "noisy"
comparison_domain="se"
exp_dict = torch.load(f'../Logs/{dataset_condition}/{dataset_condition}_{comparison_domain}_results.pt')

In [ ]:
#MIMO

# getting metrics for noisy_reverb
import os
from collections import namedtuple

metrics = namedtuple('metrics', ["mix_stoi", "enh_stoi", "mix_pesq_nb", "enh_pesq_nb", "mix_frm_Acc", "est_frm_Acc", "mix_blk_Acc", "est_blk_Acc"])


dataset_condition = "noisy_reverb"
comparison_domain = "doa"

scenarios = ["stationary", "moving"]
#t60_list = [0.2, 0.4, 0.6, 0.8, 1.0] # rows 
#snr_list = [-5.0, 0.0, 5.0, 10.0, 20.0] # col

files_list = os.listdir(f'../Logs/{dataset_condition}/diffuse_noise/miso_ri_mag')
exp_dict = {}
doa_exp_dict = {}
for file in files_list:
    if os.path.isfile(os.path.join(f'../Logs/{dataset_condition}/diffuse_noise/miso_ri_mag', file)) and os.path.splitext(file)[1]==".out":
        try:
            with open(os.path.join(f'../Logs/{dataset_condition}/diffuse_noise/miso_ri_mag', file), 'r') as f:
                for line in f.readlines():
                    if "ckpt_dir" in line.strip():
                        mic_idx, train, test, t60, snr = get_test_config_details(line.split(), dataset_condition)
                    elif "PESQ_NB_epoch" in line.strip():
                        if "PESQ_NB_epoch"==line.strip().split()[1]:
                            pesq_nb_enh = line.strip().split()[3]
                            #print(pesq_nb_enh)
                        elif "MIX_PESQ_NB_epoch"==line.strip().split()[1]:
                            pesq_nb_mix = line.strip().split()[3]
                            #print(pesq_nb_mix)
                        else:
                            continue
                        #print(line)
                    elif "STOI_epoch" in line.strip():
                        if "STOI_epoch"==line.strip().split()[1]:
                            stoi_enh = line.strip().split()[3]
                            #print(stoi_enh)
                        elif "MIX_STOI_epoch"==line.strip().split()[1]:
                            stoi_mix = line.strip().split()[3]
                            #print(stoi_mix)
                        else:
                            continue
                    elif "blk_Acc_epoch" in line.strip():
                        if "est_blk_Acc_epoch"==line.strip().split()[1]:
                            est_blk_Acc = line.strip().split()[3]
                        elif "mix_blk_Acc_epoch"==line.strip().split()[1]:
                            mix_blk_Acc = line.strip().split()[3]
                        else:
                            continue
                    elif "frm_Acc_epoch" in line.strip():
                        if "est_frm_Acc_epoch"==line.strip().split()[1]:
                            est_frm_Acc = line.strip().split()[3]
                        elif "mix_frm_Acc_epoch"==line.strip().split()[1]:
                            mix_frm_Acc = line.strip().split()[3]
                        else:
                            continue
                    
                    #print(line)
        except UnicodeDecodeError:
            print(file)
            #t60 = 1.0
            #snr = 10.0
            #train = "stationary"
            #test = "moving"
            #stoi_mix, stoi_enh, pesq_nb_mix, pesq_nb_enh, mix_frm_Acc, est_frm_Acc, mix_blk_Acc, est_blk_Acc = 0,0,0,0,0,0,0,0


    
        

        exp_metrics = metrics(stoi_mix, stoi_enh, pesq_nb_mix, pesq_nb_enh, mix_frm_Acc, est_frm_Acc, mix_blk_Acc, est_blk_Acc)

        #exp_metrics = { "mix_stoi": stoi_mix, "enh_stoi": stoi_enh, "mix_pesq_nb": pesq_nb_mix, "enh_pesq_nb": pesq_nb_enh}

        if dataset_condition =="reverb":
            exp_app_str = f't60_{t60}'
        elif dataset_condition =="noisy":
            exp_app_str = f'snr_{snr}'
        elif dataset_condition =="noisy_reverb":
            exp_app_str = f't60_{t60}_snr_{snr}'
        else:
            exp_app_str = ''

        key_str = f'train_{train}_test_{test}_{exp_app_str}' 
        exp_dict[key_str] = exp_metrics
        #doa_exp_dict[f'train_{train}_mic_idx_{mic_idx}_test_{test}_{exp_app_str}'] = doa_metrics

        #break




In [ ]:
exp_dict

In [ ]:
import torch
torch.save(exp_dict, f"{dataset_condition}_diffuse_noise_miso_ri_mag_results.pt")

In [ ]:
len(exp_dict)

In [ ]:
exp_dict['train_moving_test_stationary_t60_1.0_snr_-5.0']

In [1]:
def get_test_config_details_v2(lst, dataset_condition):
    ckpt_name_list = lst[1].split('/')
    #print(ckpt_name_list)
    loss_function, train, mic_idx = ckpt_name_list[-4], ckpt_name_list[-3], ckpt_name_list[-1][-2] 
    if "stationary"!=train:
        loss_function, train, mic_idx = ckpt_name_list[-5], ckpt_name_list[-4], ckpt_name_list[-1][-2] #

    exp_name_list = lst[-1].split('_')
    #idx = 4 if exp_name_list[-1][0]=='-' else 3
    if dataset_condition=="noisy_reverb":
        snr, t60, test = exp_name_list[-1][:-3], exp_name_list[-3], exp_name_list[-5]
    elif dataset_condition=="reverb":
        snr, t60, test = None, exp_name_list[-1][:-1], exp_name_list[-3]
    elif dataset_condition=="noisy":
        snr, t60, test = exp_name_list[-1][:-3], None, exp_name_list[-3]
    else:
        pass

    return mic_idx, train, test, t60, snr, loss_function

In [2]:
#Loss functions comparison

#MIMO

# getting metrics for noisy_reverb
import os
from collections import namedtuple

metrics = namedtuple('metrics', ["mix_stoi", "enh_stoi", "mix_pesq_nb", "enh_pesq_nb", "mix_frm_Acc", "est_frm_Acc", "mix_blk_Acc", "est_blk_Acc", "mix_2mic_frm_Acc", "est_2mic_frm_Acc"])


dataset_condition = "noisy_reverb"
noise_simulation = "point_source"  #"diffuse" #
comparison_domain = "doa"

scenarios = ["stationary"]#, "moving"]
#t60_list = [0.2, 0.4, 0.6, 0.8, 1.0] # rows 
#snr_list = [-5.0, 0.0, 5.0, 10.0, 20.0] # col

base_dir = f'../Logs/8cm_array_4mic/{dataset_condition}/{noise_simulation}/simu_rirs_doa_mask'

files_list = os.listdir(base_dir)
exp_dict = {}
doa_exp_dict = {}
for file in files_list:
    if os.path.isfile(os.path.join(base_dir, file)) and os.path.splitext(file)[1]==".out":
        try:
            with open(os.path.join(base_dir, file), 'r') as f:
                for line in f.readlines():
                    if "ckpt_dir" in line.strip():
                        mic_idx, train, test, t60, snr, loss_function = get_test_config_details_v2(line.split(), dataset_condition)
                        
                    elif "PESQ_NB_epoch" in line.strip():
                        if "PESQ_NB_epoch"==line.strip().split()[1]:
                            pesq_nb_enh = line.strip().split()[3]
                            #print(pesq_nb_enh)
                        elif "MIX_PESQ_NB_epoch"==line.strip().split()[1]:
                            pesq_nb_mix = line.strip().split()[3]
                            #print(pesq_nb_mix)
                        else:
                            continue
                        #print(line)
                    elif "STOI_epoch" in line.strip():
                        if "STOI_epoch"==line.strip().split()[1]:
                            stoi_enh = line.strip().split()[3]
                            
                        elif "MIX_STOI_epoch"==line.strip().split()[1]:
                            stoi_mix = line.strip().split()[3]
                            #print(stoi_mix)
                        else:
                            continue
                    elif "blk_Acc_epoch" in line.strip():
                        if "est_blk_Acc_epoch"==line.strip().split()[1]:
                            est_blk_Acc = line.strip().split()[3]
                        elif "mix_blk_Acc_epoch"==line.strip().split()[1]:
                            mix_blk_Acc = line.strip().split()[3]
                        else:
                            continue
                    elif "frm_Acc_epoch" in line.strip():
                        if "est_frm_Acc_epoch"==line.strip().split()[1]:
                            est_frm_Acc = line.strip().split()[3]
                        elif "mix_frm_Acc_epoch"==line.strip().split()[1]:
                            mix_frm_Acc = line.strip().split()[3]
                        elif "est_2mic_frm_Acc_epoch"==line.strip().split()[1]:
                            est_2mic_frm_Acc = line.strip().split()[3]
                        elif "mix_2mic_frm_Acc_epoch"==line.strip().split()[1]:
                            mix_2mic_frm_Acc = line.strip().split()[3]
                        else:
                            continue

                    
                    
                    #print(line)
        except UnicodeDecodeError:
            print(file)
            #t60 = 1.0
            #snr = 10.0
            #train = "stationary"
            #test = "moving"
            #stoi_mix, stoi_enh, pesq_nb_mix, pesq_nb_enh, mix_frm_Acc, est_frm_Acc, mix_blk_Acc, est_blk_Acc = 0,0,0,0,0,0,0,0

        exp_metrics = metrics(stoi_mix, stoi_enh, pesq_nb_mix, pesq_nb_enh, mix_frm_Acc, est_frm_Acc, mix_blk_Acc, est_blk_Acc, mix_2mic_frm_Acc, est_2mic_frm_Acc)

        #exp_metrics = { "mix_stoi": stoi_mix, "enh_stoi": stoi_enh, "mix_pesq_nb": pesq_nb_mix, "enh_pesq_nb": pesq_nb_enh}

        if dataset_condition =="reverb":
            exp_app_str = f't60_{t60}'
        elif dataset_condition =="noisy":
            exp_app_str = f'snr_{snr}',
        elif dataset_condition =="noisy_reverb":
            exp_app_str = f't60_{t60}_snr_{snr}'
        else:
            exp_app_str = ''

        key_str = f'{loss_function}_train_{train}_test_{test}_{exp_app_str}' 
        #print(key_str)
        exp_dict[key_str] = exp_metrics
        #doa_exp_dict[f'train_{train}_mic_idx_{mic_idx}_test_{test}_{exp_app_str}'] = doa_metrics

        #break




In [14]:
len(exp_dict)

50

In [3]:
loss_list = ["MIMO_RI", "MIMO_RI_MAG", "MIMO_RI_MAG_PD", "MIMO_RI_PD", "MIMO_RI_PD_REF"]
t60, snr = 1.0, 0.0
for _loss in loss_list:
    key_str = f'{_loss}_train_{train}_test_{test}_t60_{t60}_snr_{snr}' 
    print(exp_dict[key_str])

metrics(mix_stoi='0.5217880695024105', enh_stoi='0.7868699393087003', mix_pesq_nb='1.2924221214363807', enh_pesq_nb='1.6593604370531174', mix_frm_Acc='0.2301396280527115', est_frm_Acc='0.5540733933448792', mix_blk_Acc='0.3375861346721649', est_blk_Acc='0.8570632338523865', mix_2mic_frm_Acc='0.140549436211586', est_2mic_frm_Acc='0.4510508179664612')
metrics(mix_stoi='0.5217880695024105', enh_stoi='0.7952536103124763', mix_pesq_nb='1.2924221214363807', enh_pesq_nb='2.026929876851198', mix_frm_Acc='0.2301396280527115', est_frm_Acc='0.5499127507209778', mix_blk_Acc='0.3375861346721649', est_blk_Acc='0.8146423697471619', mix_2mic_frm_Acc='0.140549436211586', est_2mic_frm_Acc='0.4427228271961212')
metrics(mix_stoi='0.5217880695024105', enh_stoi='0.796182449776436', mix_pesq_nb='1.2924221214363807', enh_pesq_nb='2.008426424696095', mix_frm_Acc='0.2301396280527115', est_frm_Acc='0.7572082877159119', mix_blk_Acc='0.3375861346721649', est_blk_Acc='0.9060941934585571', mix_2mic_frm_Acc='0.1405494

In [ ]:
exp_dict

In [ ]:
exp_dict